In [1]:
# import necessary libraries
import speech_recognition as sr
from gtts import gTTS
import playsound
import nltk
from nltk.tokenize import word_tokenize
import csv
import os
import gradio as gr

# Download necessary NLTK data
nltk.download('punkt')

/Users/stevenhill/Desktop/bootcamp/OSU-VIRT-AI-PT-02-2024-U-LOLC/Project-3/dev/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/stevenhill/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
# Load inventory from CSV
def load_inventory(file_path):
    inventory = {}
    with open(file_path, mode='r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            product = row["product"]
            inventory[product] = {
                "quantity": int(row["quantity"]),
                "volume": row["volume"],
                "price": float(row["price"])
            }
    return inventory

# Update the file path to the resources folder
csv_file_path = os.path.join('Resources', 'final_inventory.csv')
inventory = load_inventory(csv_file_path)

In [3]:
# Create a function that will recognize when a person is speaking
def listen(audio_file_path):
    print(f"Received audio file path: {audio_file_path}")
    recognizer = sr.Recognizer()
    try:
        with sr.AudioFile(audio_file_path) as source:
            audio = recognizer.record(source)
            text = recognizer.recognize_google(audio)
            print(f"Recognized text: {text}")
            return text
    except sr.UnknownValueError:
        print("Error: Could not understand the audio")
        return "Sorry, I did not understand that."
    except sr.RequestError:
        print("Error: Could not request results from Google Speech Recognition service.")
        return "Sorry, the service is down."
    except AssertionError as e:
        print(f"AssertionError: {e}")
        return "Sorry, the audio file is not valid."

def speak(text):
    if not text:
        text = "Sorry, I have nothing to say."
    tts = gTTS(text=text, lang='en')
    filename = "response.mp3"
    tts.save(filename)
    try:
        playsound.playsound(filename)
        print(f"Playing sound: {filename}")
    except Exception as e:
        print(f"Error playing sound: {e}")
    return filename

In [4]:

def process_command(command):
    print(f"Processing command: {command}")
    tokens = word_tokenize(command.lower())
    product_name = extract_product_name(command)
    
    if "find" in tokens and "product" in tokens:
        response = "Sure, I can help you find a product. What are you looking for?"
    elif "check" in tokens and "stock" in tokens:
        product_name = extract_product_name(command)
        if product_name in inventory:
            item = inventory[product_name]
            response = f"We have {item['quantity']} {product_name}(s) in stock, each {item['volume']} at ${item['price']}."
        else:
            response = f"Sorry, we don't have {product_name} in our inventory."
    elif "price" in tokens:
        product_name = extract_product_name(command)
        if product_name in inventory:
            response = f"The price of {product_name} is ${inventory[product_name]['price']} per {inventory[product_name]['volume']}."
        else:
            response = f"Sorry, we don't have {product_name} in our inventory."
    elif "volume" in tokens:
        product_name = extract_product_name(command)
        if product_name in inventory:
            response = f"The volume of {product_name} is {inventory[product_name]['volume']}."
        else:
            response = f"Sorry, we don't have {product_name} in our inventory."
    elif "information" in tokens and "paint" in tokens:
        response = "I can provide information about our stock and pricing. What would you like to know?"
    else:
        response = "I'm sorry, I didn't understand that. Can you please repeat?"
    print(f"Response: {response}")
    return response

def extract_product_name(command):
    command = command.lower()
    best_match = ""
    best_match_length = 0
    for product in inventory.keys():
        product_lower = product.lower()
        if product_lower in command:
            if len(product_lower) > best_match_length:
                best_match = product
                best_match_length = len(product_lower)
    return best_match



In [5]:
def assistant(audio_file_path):
    print(f"Assistant received audio file path: {audio_file_path}")
    command = listen(audio_file_path)
    response = process_command(command)
    audio_response = speak(response)
    return response, audio_response

In [6]:
# Create Gradio interface
iface = gr.Interface(
    fn=assistant,
    inputs=gr.Audio("microphone", type="filepath"),
    outputs=[gr.Textbox(), gr.Audio(type="filepath")]
)

iface.launch(share=True)

Running on local URL:  http://127.0.0.1:7861
Running on public URL: https://f22335db86254e199e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Assistant received audio file path: /private/var/folders/xw/v1jvfgnx0qd3b1ytj3y03tk00000gn/T/gradio/de4071ce17fc06af090e8573ca4f60a76143c07c/audio.wav
Received audio file path: /private/var/folders/xw/v1jvfgnx0qd3b1ytj3y03tk00000gn/T/gradio/de4071ce17fc06af090e8573ca4f60a76143c07c/audio.wav
Recognized text: what is the price of Regal select flat white
Processing command: what is the price of Regal select flat white
Response: Sorry, we don't have  in our inventory.
Playing sound: response.mp3
Assistant received audio file path: /private/var/folders/xw/v1jvfgnx0qd3b1ytj3y03tk00000gn/T/gradio/b59baf0eb02bbf426691453aee6b8386ccb35c77/audio.wav
Received audio file path: /private/var/folders/xw/v1jvfgnx0qd3b1ytj3y03tk00000gn/T/gradio/b59baf0eb02bbf426691453aee6b8386ccb35c77/audio.wav
Recognized text: what is the price of Regal select flat base 1
Processing command: what is the price of Regal select flat base 1
Response: Sorry, we don't have  in our inventory.
Playing sound: response.mp3
